# Peer-graded Assignment: Эксперименты с моделью

На прошлой неделе вы поучаствовали в соревновании на kaggle и, наверняка, большинство успешно справилось с прохождением baseline, а значит пора двигаться дальше - заняться оптимизацией модели, провести серию экспериментов и построить сильное финальное решения.

В этом задании вам нужно провести ряд эскпериментов, оценить качество полученных в процессе экспериментирования моделей и выбрать лучшее решение.

In [30]:
import pandas as pd
import numpy as np

import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import FunctionTransformer
from sklearn import metrics
from catboost import CatBoostClassifier
from sklearn.model_selection import learning_curve
import xgboost as xgb
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV

import plotly.express as px

## Инструкции

0. Подготовка датасета

In [2]:
df = pd.read_csv('orange_small_churn_train_data.csv', index_col=['ID'])
df['labels'].fillna(0, inplace=True)
labels, train = df['labels'], df.drop(columns=['labels'])
labels.replace({-1: 0}, inplace=True)
labels = labels.values

In [3]:
percent_missing = train.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing}).sort_values('percent_missing', ascending=False)

In [4]:
train = train[list(missing_value_df[missing_value_df.percent_missing < 50.].index)]
train.head()

,Var72,Var94,Var126,Var109,Var149,Var24,Var144,Var81,Var206,Var6,...,Var73,Var113,Var212,Var193,Var210,Var207,Var204,Var196,Var195,Var198
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,4.0,144.0,389396.0,20.0,9.0,14599.92,IYzP,3052.0,...,34,-1209960.0,JBfYVit4g8,AERks4l,uKAI,GjJ35utlTa_GNSvxxpb9ju,k13i,1K8T,taul,UaKK0yW
1,3.0,32289.0,40.0,80.0,735.0,2.0,18.0,67529.09,haYg,1813.0,...,128,417932.0,XfqtO3UdzaXh_,2Knk1KF,uKAI,me75fM6ugJ,FbIm,1K8T,taul,Bnunsla
2,3.0,53388.0,36.0,40.0,0.0,0.0,27.0,85266.00,hAFG,1953.0,...,166,-124655.2,4kVnq_T26xq1p,LrdZy8QqgUfkVShG,uKAI,7M47J5GA0pTYIFxg5uy,mTeA,1K8T,taul,fhk21Ss
3,NaN,NaN,NaN,32.0,0.0,0.0,0.0,74107.20,IYzP,1533.0,...,30,378473.6,NhsEn4L,RO12,uKAI,me75fM6ugJ,vzJD,1K8T,taul,uoZk2Zj
4,3.0,106455.0,-28.0,32.0,554414.0,2.0,9.0,171072.90,zm5i,686.0,...,32,142602.4,NhsEn4L,RO12,uKAI,me75fM6ugJ,m_h1,1K8T,taul,kugYdIL


In [5]:
def fillna_imputer(df):
    '''
    Заменяет Nan значения на строковый маркер 'Nan' и добавляет boolean столбец для каждого признака
    '''
    column_names = df.columns.to_list()
    result = pd.DataFrame()
    for name in column_names:
        result[name] = df[name].fillna(value = 'missing')
        result[name + '_bool'] = df[name].isna()
    return result.reset_index(drop = True)

bool_fillna_imputer = FunctionTransformer(fillna_imputer, validate=False)

In [37]:
numeric_features = list(set([f'Var{i}' for i in range(1, 191)]).intersection(set(train.columns)))
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('variancethreshold', VarianceThreshold(0.1)),
    ('scaler', StandardScaler())])

categorical_features = list(set([f'Var{i}' for i in range(191, 230)]).intersection(set(train.columns)))
categorical_transformer = Pipeline(steps=[
    ('imputer', bool_fillna_imputer),
    ('onehot', ce.CatBoostEncoder(random_state=0)),
    ('variancethreshold', VarianceThreshold(0.0))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [38]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RidgeClassifier(random_state=0, class_weight='balanced'))])

1\. Начнем с простого. Давайте оценим как много объектов действительно нужно для построения качественной модели. Для обучения доступна достаточно большая выборка и может так оказаться, что начиная с некоторого момента рост размера обучающей выборки перестает влиять на качество модели. Постройте кривые обучения, обучая модель на выборках разного размера начиная с небольшого количество объектов в обучающей выборке и постепенно наращивая её размер с некоторым шагом. Обратите внимание на `sklearn.model_selection.learning_curve`

In [39]:
train_size_abs, train_scores, test_scores = learning_curve(
    clf, train, labels, train_sizes=[0.3, 0.6, 0.9], shuffle=True, random_state=0
)
for train_size, cv_train_scores, cv_test_scores in zip(
        train_size_abs, train_scores, test_scores
):
    print(f"{train_size} samples were used to train the model")
    print(f"The average train accuracy is {cv_train_scores.mean():.2f}")
    print(f"The average test accuracy is {cv_test_scores.mean():.2f}")

4391 samples were used to train the model
The average train accuracy is 0.63
The average test accuracy is 0.62
8783 samples were used to train the model
The average train accuracy is 0.63
The average test accuracy is 0.61
13175 samples were used to train the model
The average train accuracy is 0.63
The average test accuracy is 0.61


In [40]:
cross_val_score(clf, train, labels, scoring='balanced_accuracy', cv=5, n_jobs=2)

array([0.60832819, 0.62537129, 0.64572412, 0.64158058, 0.60779658])

2\. Примените к выборке технологию undersampling: для этого нужно убрать из обучения некоторое количество объектов большего класса таким образом, чтобы соотношение классов изменилось. Попробуйте не менее трёх различных вариантов undersampling (варианты могут отличаться как по количество отфильтрованных объектов, так и по принципу выборка объектов для отсеивания из выборки). Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

In [41]:
rus = RandomUnderSampler(random_state=0)
train_resampled, labels_resampled = rus.fit_resample(train, labels)

In [42]:
train_size_abs, train_scores, test_scores = learning_curve(
    clf, train_resampled, labels_resampled, train_sizes=[0.3, 0.6, 0.9], shuffle=True, random_state=0
)
for train_size, cv_train_scores, cv_test_scores in zip(
        train_size_abs, train_scores, test_scores
):
    print(f"{train_size} samples were used to train the model")
    print(f"The average train accuracy is {cv_train_scores.mean():.2f}")
    print(f"The average test accuracy is {cv_test_scores.mean():.2f}")

660 samples were used to train the model
The average train accuracy is 0.67
The average test accuracy is 0.59
1321 samples were used to train the model
The average train accuracy is 0.67
The average test accuracy is 0.61
1982 samples were used to train the model
The average train accuracy is 0.68
The average test accuracy is 0.62


In [43]:
cross_val_score(clf, train_resampled, labels_resampled, scoring='balanced_accuracy', cv=5, n_jobs=2)

array([0.5, 0.5, 0.5, 0.5, 0.5])

3\. Теперь перейдем к работе с признаками. Ранее вы реализовали несколько стратегий для обработки пропущенных значений. Сравните эти стратегии между собой с помощью оценки качества моделей кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка пропущенных значений сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

Делал на прошлой неделе. Лучшая обработка в пайплайне

4\. Также вы уже реализовали несколько стратегий для обработки категориальных признаков. Сравните эти стратегии между собой с помощью оценки качества моделей по кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка категориальных признаков сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

Делал на прошлой неделе. Лучшая обработка в пайплайне

5\. Все ли признаки оказались полезными для построения моделей? Проведите процедуру отбора признаков, попробуйте разные варианты отбора (обратите внимание на модуль `sklearn.feature_selection`). Например, можно выбрасывать случайные признаки или строить отбор на основе l1-регуляризации - отфильтровать из обучения признаки, которые получат нулевой вес при построении регрессии с l1-регуляризацией (`sklearn.linear_model.Lasso`). И всегда можно придумать что-то своё=) Попробуйте как минимум 2 различные стратегии, сравните результаты. Помог ли отбор признаков улучшить качество модели? Поясните свой ответ.

In [49]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('selector', SelectKBest(f_classif, k=70)),
                      ('classifier', RidgeClassifier(random_state=0, class_weight='balanced'))])

cross_val_score(clf, train, labels, scoring='balanced_accuracy', cv=5, n_jobs=2)

array([0.62287364, 0.62133745, 0.64720167, 0.63913943, 0.60642166])

6\. Подберите оптимальные параметры модели. Обратите внимание, что в зависимости от того, как вы обработали исходные данные, сделали ли балансировку классов, сколько объектов оставили в обучающей выборке и др. оптимальные значения параметров могут меняться. Возьмите наилучшее из ваших решений на текущий момент и проведите процедуру подбора параметров модели (обратите внимание на `sklearn.model_selection.GridSearchCV`) Как подбор параметров повлиял на качество модели?

In [50]:
parameters = {'classifier__alpha':[0.3, 0.5, 1.]}
grid = GridSearchCV(clf, parameters)
grid.fit(train, labels)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('variancethreshold',
                                                                                          VarianceThreshold(threshold=0.1)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Var44',
                                                                          'Var13',
                         

In [52]:
grid.cv_results_

{'mean_fit_time': array([0.87957358, 0.78598628, 0.78994784]),
 'std_fit_time': array([0.12553363, 0.08442278, 0.09005105]),
 'mean_score_time': array([0.14152074, 0.12309847, 0.12329168]),
 'std_score_time': array([0.03357628, 0.00636611, 0.00534279]),
 'param_classifier__alpha': masked_array(data=[0.3, 0.5, 1.0],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'classifier__alpha': 0.3},
  {'classifier__alpha': 0.5},
  {'classifier__alpha': 1.0}],
 'split0_test_score': array([0.62595628, 0.62650273, 0.62377049]),
 'split1_test_score': array([0.65409836, 0.65409836, 0.65491803]),
 'split2_test_score': array([0.62595628, 0.62622951, 0.62759563]),
 'split3_test_score': array([0.63224044, 0.63142077, 0.63114754]),
 'split4_test_score': array([0.6373326 , 0.63569281, 0.63350642]),
 'mean_test_score': array([0.63511679, 0.63478884, 0.63418762]),
 'std_test_score': array([0.01040493, 0.01026587, 0.01087591]),
 'rank_test_score': arra

7\. Предложите методику оценки того, какие признаки внесли наибольший вклад в модель (например, это могут быть веса в случае регрессии, а также большое количество моделей реализуют метод `feature_importances_` - оценка важности признаков). На основе предложенной методики проанализируйте, какие признаки внесли больший вклад в модель, а какие меньший?

In [56]:
clf.fit(train, labels)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('variancethreshold',
                                                                   VarianceThreshold(threshold=0.1)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Var44', 'Var13', 'Var153',
                                                   'Var112', 'Var73', 'Var173',
                                                   'Var113', 'Var7', 'Var21',
                                                   'Var6', 'Var163', 'Var74',
                                                   'Var76', 

In [59]:
clf.named_steps['classifier'].coef_

array([[ 0.00873703, -0.06936441, -0.01672111, -0.20649621,  0.06872468,
        -0.01982019,  0.06150716, -0.02960969,  0.01730626, -0.04536996,
         0.01068546, -0.0143369 , -0.01289121, -0.09404157,  0.00826623,
        -0.00463342, -0.02755816,  0.0288069 ,  0.01084958, -0.0096769 ,
         0.02868448,  0.02723469,  0.03039234,  0.03875166, -0.0135567 ,
         0.00783542,  0.02755343,  0.01398893,  0.13695176, -0.00810332,
        -0.03249959,  0.01408752, -0.03398342,  0.02326126,  0.03361983,
         0.0208979 ,  0.04927431,  0.01009069,  0.43307198, -0.09681323,
        -0.07663755, -0.82860438, -0.89036375, -0.15872452, -0.03777044,
         0.06679433, -1.21423399,  0.22223294, -0.15872452,  0.93785   ,
         0.51889146,  0.36587896,  2.9733062 , -1.11172649, -0.3402967 ,
         0.63123472, -0.15872452, -0.90870616, -0.22496969,  2.23703048,
        -0.04369948, -0.09681323,  0.92323674,  0.30202941,  0.03612384,
         1.14574887, -0.03777044,  2.45751935,  0.0